In [17]:
# ORIGINAL CODE : https://machinelearningmastery.com/weighted-average-ensemble-with-python/
# By Jason Brownlee - MAY 2021

# Modifed by MAHAYASA ADIPUTRA - OCT 2022
import time
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix ,accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as mt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import VotingClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
le = LabelEncoder()
start_time = time.time()
# get the dataset
# Importing the dataset
dataset=pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/club-churn.csv')
#filling missing value with mean
dataset['points_in_wallet'].fillna(value=dataset['points_in_wallet'].mean(), inplace=True)
dataset['avg_frequency_login_days'] = pd.to_numeric(dataset['avg_frequency_login_days'], errors='coerce').fillna(0)
#transform data into numeric value
dataset['gender']=le.fit_transform(dataset['gender'])
dataset['region_category']=le.fit_transform(dataset['region_category'])
dataset['membership_category']=le.fit_transform(dataset['membership_category'])
dataset['joined_through_referral']=le.fit_transform(dataset['joined_through_referral'])
dataset['preferred_offer_types']=le.fit_transform(dataset['preferred_offer_types'])
dataset['medium_of_operation']=le.fit_transform(dataset['medium_of_operation'])
dataset['internet_option']=le.fit_transform(dataset['internet_option'])
dataset['used_special_discount']=le.fit_transform(dataset['used_special_discount'])
dataset['offer_application_preference']=le.fit_transform(dataset['offer_application_preference'])
dataset['past_complaint']=le.fit_transform(dataset['past_complaint'])
dataset['complaint_status']=le.fit_transform(dataset['complaint_status'])
dataset['feedback']=le.fit_transform(dataset['feedback'])

dataset=dataset.dropna()

X=dataset.drop(['churn_risk_score','security_no','joining_date','referral_id','last_visit_time'],axis=1)
y=dataset["churn_risk_score"]

#imblearn SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)
#smote_enn = SMOTEENN()
#smote_enn = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
#X, y = smote_enn.fit_resample(X, y)
ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
X, y = ncr.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)
# get a list of base models
def get_models():
	models = list()
	models.append(('xb', XGBClassifier(colsample_bytree=0.6, max_depth = 5, gamma=1.5, min_child_weight=5)))
	models.append(('rf', RandomForestClassifier(n_estimators =100, criterion = 'gini', random_state = 0)))
#	models.append(('ab', AdaBoostClassifier(n_estimators=100,learning_rate=0.1,algorithm='SAMME.R')))
	return models
 
# evaluate each base model
def evaluate_models(models, X,y):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# evaluate the model
		acc=cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
		# store the performance
		scores.append(mean(acc))
		# report model performance
	return scores



# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X,y)
print(scores)
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='soft', weights=scores)
score=cross_val_score(ensemble, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

[0.9812507253461356, 0.9804347852887456]
F1 score: 0.941
STD F1 Score: 0.003


In [18]:
rc=cross_val_score(ensemble, X, y, scoring='recall', cv=cv, n_jobs=-1)
rc=mean(rc)
rc

0.9527738188763527

In [19]:
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(ensemble, X, y, cv=5, scoring=scorer)
sp=mean(sp)
sp

0.9366377890245484

In [20]:
auc=cross_val_score(ensemble, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
auc=mean(auc)
auc

0.9816649424959643

In [21]:
import math

gmean=rc*sp
sqrtg = math.sqrt(gmean)
sqrtg

0.9446713519276545